<a href="https://colab.research.google.com/github/NShravanReddy/SatvickRecipe/blob/master/satvick_recipe_model_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this code add your Hugging face token key in secrets thats it

In [1]:
!pip install gradio
!pip install bitsandbytes>=0.39.0
!pip install --upgrade accelerate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 104.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.3.0
    Uninstalling accelerate-1.3.0:
      Successfully uninstalled accelerate-1.3.0


In [2]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

# Load the model and tokenizer
model_name = "Nadiveedishravanreddy/satvick_lora_model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Define the prediction function
def get_preparation_steps(recipe_name):
    """
    Generate preparation steps for a given recipe name.
    """
    # Format the input prompt
    prompt = f"Recipe: {recipe_name}\nPreparation Steps:"

    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Stream text generation
    streamer = TextStreamer(tokenizer)
    print("Generating response...")
    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=128)

    # Generate and decode the response
    outputs = model.generate(inputs.input_ids, max_length=256, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response[len(prompt):].strip()  # Exclude the prompt from the output

# Create the Gradio Interface
interface = gr.Interface(
    fn=get_preparation_steps,
    inputs=gr.Textbox(label="Enter Recipe Name", placeholder="E.g., Ragi Idiyappam"),
    outputs=gr.Textbox(label="Preparation Steps"),
    title="Recipe Chatbot",
    description="Enter a recipe name to get the preparation steps generated by the Satvick Lora Model.",
    theme="compact"
)

# Launch the interface
if __name__ == "__main__":
    interface.launch()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1102: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-67dfd741-4b94ac2a2c2c02b11e008961;77ac5cd7-0a46-410d-8c3b-dd3c3d3de720)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b3732d1bab67eb772e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
# Count the number of parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}")

Total Parameters: 4,582,543,360
Trainable Parameters: 0


In [ ]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

# Load the model and tokenizer on CPU
model_name = "Nadiveedishravanreddy/satvick_lora_model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cpu")  # Explicitly specify CPU

def get_preparation_steps(recipe_name):
    """
    Generate preparation steps for a given recipe name.
    """
    # Format the input prompt
    prompt = f"Recipe: {recipe_name}\nPreparation Steps:"

    # Tokenize the input on CPU
    inputs = tokenizer(prompt, return_tensors="pt").to("cpu")  # Use CPU

    # Stream text generation
    streamer = TextStreamer(tokenizer)
    print("Generating response...")
    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=128)

    # Generate and decode the response
    outputs = model.generate(inputs.input_ids, max_length=256, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response[len(prompt):].strip()  # Exclude the prompt from the output

# Create the Gradio Interface
interface = gr.Interface(
    fn=get_preparation_steps,
    inputs=gr.Textbox(label="Enter Recipe Name", placeholder="E.g., Ragi Idiyappam"),
    outputs=gr.Textbox(label="Preparation Steps"),
    title="Recipe Chatbot",
    description="Enter a recipe name to get the preparation steps generated by the Satvick Lora Model.",
    theme="compact"
)

# Launch the interface
if __name__ == "__main__":
    interface.launch()

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1048: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-678bbac7-5164c2be0c59bbe72deff338;2eb5b9b1-3e87-4d56-b489-fc8146ff8b74)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b833d485a5440b76be.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
